In [ ]:
#@title Customize your settings then run the cell { vertical-output: true }
#@markdown Choose a model
Model = "7th Layer" #@param ["7th Layer", "8528 Diffusion", "A Certain Model", "A Certain Thing", "A Certainty", "Abyss Orange Mix 2", "Abyss Orange Mix 3", "All In One Pixel Model", "Analog Diffusion", "Anything V3", "Anything V4", "Anything Gape", "Arcane Diffusion 3", "Babes Blend", "Berry Mix 1.5", "Cafe Instagram v6", "Chillout Mix", "Cinematic Diffusion", "Color Bomb Mix", "Comic Diffusion v2", "Comics Blend", "Complex Lineart", "ControlNet", "Cool Japan Diffusion 2.1", "Counterfeit v2.0", "Cyberpunk Anime Diffusion", "Deliberate", "Dpepmkmp", "Dream Like Sam Kuvshino", "Dreamlike Diffusion 1", "Dreamlike Photoreal 2", "Dreamshaper 3.32", "DucHaitenAIArt v1.1", "Eimis Anime Diffusion 1", "Elldreth's Lucid Mix v10", "Elysium Anime 2", "F222", "Food Crit", "Grapefruit", "HasDX", "Hassan Blend 1.4", "Illuminati v1.0", "Illuminati v1.1", "Illustration Diffusion", "Inkpunk", "Izumi v1", "Knollingcase", "Midjourney v4", "Mo Di Diffusion", "Monstermash Any 3", "Mousey Mix", "Neverending Dream", "Open Anime Journey", "Open Journey v2", "Papercut Diffusion", "Pastel Mix Stylized Anime", "Photorealostic Fuen v1", "Pix2Pix", "Pony Diffusion 2", "Popup Book Diffusion", "Protogen v2.2", "Protogen X 3.4", "Protogen Nova", "Rando mix 3", "Riffusion v1", "RPG v4", "SamDoesArt Diffusion", "SamDoesSexy Blend", "Seek Art Mega v1", "Simpmaker 3k", "Stable Diffusion 1.4", "Stable Diffusion 1.5", "Stable Diffusion 1.5 Inpainting", "Stable Diffusion 2", "Stable Diffusion 2 Base", "Stable Diffusion 2 Depth", "Stable Diffusion 2 Inpainting", "Stable Diffusion 2.1", "Stable Diffusion 2.1 Base", "Synthwave Punk v2", "Uber Realistic v1", "Van Gogh Diffusion", "Vinte Protogen Mix", "Vintedois Diffusion", "Waifu Diffusion", "Waifu Diffusion 1.4", "Waifu Diffusion 1.5", "Woolitize", "Yiffy E18", "Yiffy Mix", "Yohan Diffusion"]
#@markdown Common negative embeddings
bad_artist = False #@param {type:"boolean"} 
bad_artist_anime = False #@param {type:"boolean"}
bad_hands_5 = False #@param {type:"boolean"}
bad_prompt = False #@param {type:"boolean"}
bad_prompt_version2 = False #@param {type:"boolean"}
#@markdown Common VAE
vae_ft_mse_840000_ema_pruned = True #@param {type:"boolean"}
Anything_V3_vae= False #@param {type:"boolean"}
#@markdown Choose a service to host the web-ui
Tunnel = "Google" #@param ["Google", "Cloudflare", "Localhostrun", "Remote.moe"] 
#@markdown Uncheck to launch webui without dark mode
dark_mode = True #@param {type:'boolean'}

## Adding tunnel argument

if Tunnel=="Google":
    import os
    from google.colab.output import eval_js
    os.environ['colab_url'] = eval_js("google.colab.kernel.proxyPort(7860), {'cache': false}")
    host = "--googleusercontent"
elif Tunnel=="Cloudflare":
    host = "--cloudflared"
elif Tunnel=="Localhostrun":
    host = "--localhostrun"
elif Tunnel=="Remote.moe":
    host = "--remotemoe"
else:
    host = "--googleusercontent"

## Setting dark mode on/off
if dark_mode:
    dark = "--dark theme"

## Daviljoe193 fix

!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

## Common code

!apt-get -y install -qq aria2
!pip install -q --pre xformers
!pip install -q --pre triton

!git clone -b v2.0 https://github.com/camenduru/stable-diffusion-webui
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O /content/stable-diffusion-webui/scripts/run_n_times.py
!git clone https://github.com/deforum-art/deforum-for-automatic1111-webui /content/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git clone https://github.com/camenduru/stable-diffusion-webui-huggingface /content/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
!git clone -b v2.0 https://github.com/camenduru/sd-civitai-browser /content/stable-diffusion-webui/extensions/sd-civitai-browser
!git clone https://github.com/kohya-ss/sd-webui-additional-networks /content/stable-diffusion-webui/extensions/sd-webui-additional-networks
!git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/sd-webui-controlnet
!git clone https://github.com/camenduru/openpose-editor /content/stable-diffusion-webui/extensions/openpose-editor
!git clone https://github.com/camenduru/sd-webui-tunnels /content/stable-diffusion-webui/extensions/sd-webui-tunnels
%cd /content/stable-diffusion-webui
!git reset --hard

## Downloading ControlNet OpenPose stuff

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_canny-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_depth-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_hed-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_mlsd-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_normal-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_openpose-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_scribble-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_seg-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/hand_pose_model.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose -o hand_pose_model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/body_pose_model.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose -o body_pose_model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/dpt_hybrid-midas-501f0c75.pt -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/midas -o dpt_hybrid-midas-501f0c75.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_large_512_fp32.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_large_512_fp32.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_tiny_512_fp32.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_tiny_512_fp32.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/network-bsds500.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/hed -o network-bsds500.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/upernet_global_small.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/uniformer -o upernet_global_small.pth

## Downloading requested model

if Model=="7th Layer":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/syaimu/7th_Layer/resolve/main/7th_layer/Abyss_7th_layer.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o Abyss_7th_layer.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/syaimu/7th_Layer/resolve/main/7th_anime_v3/7th_anime_v3_C.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o 7th_anime_v3_C.safetensors
elif Model=="8528 Diffusion":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/8528-diffusion/resolve/main/8528d-final.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o 8528d-final.ckpt
elif Model=="A Certain Model":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ACertainModel/resolve/main/ACertainModel.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o ACertainModel.ckpt
elif Model=="A Certain Thing":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ACertainThing/resolve/main/ACertainThing.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o ACertainThing.ckpt
elif Model=="A Certainty":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ACertainty/resolve/main/ACertainty.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o ACertainty.ckpt
elif Model=="Abyss Orange Mix 2":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/OrangeMixs/resolve/main/AbyssOrangeMix2_nsfw.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AbyssOrangeMix2_nsfw.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/OrangeMixs/resolve/main/AbyssOrangeMix2_hard.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AbyssOrangeMix2_hard.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/OrangeCocoaMix/resolve/main/OrangeCocoaMix2_hard.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o OrangeCocoaMix2_hard.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/anything-v3.0/resolve/main/Anything-V3.0.vae.pt -d /content/stable-diffusion-webui/models/Stable-diffusion -o AbyssOrangeMix2_nsfw.vae.pt
elif Model=="Abyss Orange Mix 3":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AOM3.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AOM3A1.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A2.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AOM3A2.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o AOM3A3.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -d /content/stable-diffusion-webui/models/Stable-diffusion -o orangemix.vae.pt
elif Model=="All In One Pixel Model":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/All-In-dne-Pixel-Model/resolve/main/Public-Prompts-Pixel-Model.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o Public-Prompts-Pixel-Model.ckpt
elif Model=="Analog Diffusion":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/Analog-Diffusion/resolve/main/analog-diffusion-1.0.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o analog-diffusion-1.0.ckpt
elif Model=="Anything V3":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/anything-v3.0/resolve/main/Anything-V3.0-pruned.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o Anything-V3.0-pruned.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/anything-v3.0/resolve/main/Anything-V3.0.vae.pt -d /content/stable-diffusion-webui/models/Stable-diffusion -o Anything-V3.0-pruned.vae.pt
elif Model=="Anything V4":
   #!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/anything-v4.0/resolve/main/anything-v4.0-pruned.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o anything-v4.0-pruned.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/anything-v4.0/resolve/main/anything-v4.5-pruned.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o anything-v4.5-pruned.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/anything-v4.0/resolve/main/anything-v4.0.vae.pt -d /content/stable-diffusion-webui/models/Stable-diffusion -o anything-v4.5-pruned.vae.pt
elif Model=="Anything Gape":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/AnyGape/resolve/main/AnythingGape-fp16.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o AnythingGape-fp16.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/anything-v3.0/resolve/main/Anything-V3.0.vae.pt -d /content/stable-diffusion-webui/models/Stable-diffusion -o AnythingGape-fp16.vae.pt
elif Model=="Arcane Diffusion 3":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/Arcane-Diffusion/resolve/main/arcane-diffusion-v3.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o arcane-diffusion-v3.ckpt
elif Model=="Babes Blend":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/babes/resolve/main/babes_11.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o babes_11.safetensors
elif Model=="Berry Mix 1.5":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/rbm/resolve/main/rbm.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o rbm.ckpt
elif Model=="Cafe Instagram v6":           
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/cafe-instagram-sd-1-5-v6/resolve/main/instagram-latest-plus-clip-v6e1_50000.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o instagram-latest-plus-clip-v6e1_50000.safetensors
elif Model=="Chillout Mix":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/11745 -d /content/stable-diffusion-webui/models/Stable-diffusion -o chilloutmix_NiPrunedFp32Fix.safetensors
elif Model=="Cinematic Diffusion":              
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/Cinematic-Diffusion/resolve/main/syberart.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o syberart.ckpt
elif Model=="Color Bomb Mix":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/mocker/KaBoom/resolve/main/ColorBombMix-fp16-no-ema.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o ColorBombMix-fp16-no-ema.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/mocker/KaBoom/resolve/main/FaceBombMix-fp16-no-ema.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o FaceBombMix-fp16-no-ema.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/mocker/KaBoom/resolve/main/HyperBombMix-fp16-no-ema.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o HyperBombMix-fp16-no-ema.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o ColorBombMix-fp16-no-ema.vae.pt
elif Model=="Comic Diffusion v2":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/Comic-Diffusion/resolve/main/comic-diffusion-V2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o comic-diffusion-V2.ckpt
elif Model=="Comics Blend":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/comicsblend/resolve/main/comicsblend_V1.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o comicsblend_V1.ckpt
elif Model=="Complex Lineart":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/Complex-Lineart/resolve/main/ComplexLA%20Style.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o ComplexLA%20Style.ckpt
elif Model=="ControlNet":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/mo-di-diffusion/resolve/main/moDi-v1-pruned.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o moDi-v1-pruned.ckpt
elif Model=="Cool Japan Diffusion 2.1":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/cool-japan-diffusion-2-1-0/resolve/main/v2-1-0.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o v2-1-0.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/cool-japan-diffusion-2-1-0/raw/main/v2-1-0.yaml -d /content/stable-diffusion-webui/models/Stable-diffusion -o v2-1-0.yaml
elif Model=="Counterfeit v2.0":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/Counterfeit-V2.0/resolve/main/Counterfeit-V2.0fp16.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o Counterfeit-V2.0fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/anything-v3.0/resolve/main/Anything-V3.0.vae.pt -d /content/stable-diffusion-webui/models/Stable-diffusion -o Anything-V3.0.vae.pt
elif Model=="Cyberpunk Anime Diffusion":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/Cyberpunk-Anime-Diffusion/resolve/main/Cyberpunk-Anime-Diffusion.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o Cyberpunk-Anime-Diffusion.ckpt
elif Model=="Deliberate":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/XpucT/Deliberate/resolve/main/Deliberate.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o Deliberate.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/XpucT/Deliberate/resolve/main/Deliberate_v2.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o Deliberate_v2.safetensors
elif Model=="Dpepmkmp":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/closertodeath/dpepmkmp/resolve/main/dpepmkmp.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o dpepmkmp.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o dpepmkmp.vae.pt
elif Model=="Dream Like Sam Kuvshino":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/dreamlikesamkuvshinov/resolve/main/dreamlikesamkuvshino.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o dreamlikesamkuvshino.safetensors
elif Model=="Dreamlike Diffusion 1":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/dreamlike-diffusion-1.0/resolve/main/dreamlike-diffusion-1.0.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o dreamlike-diffusion-1.0.ckpt
elif Model=="Dreamlike Photoreal 2":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/dreamlike-photoreal-2.0/resolve/main/dreamlike-photoreal-2.0.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o dreamlike-photoreal-2.0.safetensors
elif Model=="Dreamshaper 3.32":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Lykon/DreamShaper/resolve/main/Dreamshaper_3.32_baked_vae_clip_fix.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o Dreamshaper_3.32_baked_vae_clip_fix.safetensors
elif Model=="DucHaitenAIArt v1.1":
    #!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/DucHaitenAIart_v1.1a/resolve/main/DucHaitenAIart_v1.1a_emaonly.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o DucHaitenAIart_v1.1a_emaonly.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/DucHaiten/DucHaitenAIart/resolve/main/DucHaitenAIart_v2.0.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o DucHaitenAIart_v2.0.safetensors
    #!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/DucHaiten/DucHaitenDarkside/resolve/main/DucHaitenDarkside_v1.0.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o DucHaitenDarkside_v1.0.safetensors
elif Model=="Eimis Anime Diffusion 1":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/EimisAnimeDiffusion_1.0v/resolve/main/EimisAnimeDiffusion_1-0v.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o EimisAnimeDiffusion_1-0v.ckpt
elif Model=="Elldreth's Lucid Mix v10":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/elldrethSLucidMix_v10/resolve/main/elldrethSLucidMix_v10.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o elldrethSLucidMix_v10.ckpt
elif Model=="Elysium Anime 2":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/SD-Elysium-Model/resolve/main/Elysium_Anime_V2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o Elysium_Anime_V2.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/SD-Elysium-Model/resolve/main/Elysium_Anime_V3.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o Elysium_Anime_V3.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/waifu-diffusion-v1-4/resolve/main/kl-f8-anime2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o Elysium_Anime_V2.vae.pt
elif Model=="F222":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/f2/resolve/main/f2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o f2.ckpt
elif Model=="Food Crit":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/food-crit/resolve/main/food_crit.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o food_crit.ckpt
elif Model=="Grapefruit":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/grapefruit/resolve/main/grapefruit.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o grapefruit.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/grapefruit/resolve/main/grapefruit.vae.pt -d /content/stable-diffusion-webui/models/Stable-diffusion -o grapefruit.vae.pt
elif Model=="HasDX":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/hasdx/resolve/main/hasdx_emaonly.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o hasdx_emaonly.ckpt
elif Model=="Hassan Blend 1.4":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/hassanblend1.4/resolve/main/HassanBlend1.4-Pruned.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o HassanBlend1.4-Pruned.ckpt
elif Model=="Illuminati v1.0":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/IlluminatiAI/Illuminati_Diffusion_v1.0/resolve/main/illuminati_diffusion_v1.0.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o illuminati_diffusion_v1.0.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/IlluminatiAI/Illuminati_Diffusion_v1.0/raw/main/illuminati_diffusion_v1.0.yaml -d /content/stable-diffusion-webui/models/Stable-diffusion -o illuminati_diffusion_v1.0.yaml
elif Model=="Illuminati v1.1":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/13259 -d /content/stable-diffusion-webui/models/Stable-diffusion -o illuminatiDiffusionV1_v11.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/IlluminatiAI/Illuminati_Diffusion_v1.0/raw/main/illuminati_diffusion_v1.0.yaml -d /content/stable-diffusion-webui/models/Stable-diffusion -o illuminatiDiffusionV1_v11.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/15927 -d /content/stable-diffusion-webui/embeddings -o nrealfixer.pt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/15926 -d /content/stable-diffusion-webui/embeddings -o nartfixer.pt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/15921 -d /content/stable-diffusion-webui/embeddings -o nfixer.pt
elif Model=="Illustration Diffusion":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/Illustration-Diffusion/resolve/main/hollie-mengert.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o hollie-mengert.ckpt
elif Model=="Inkpunk":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/Inkpunk-Diffusion/resolve/main/Inkpunk-Diffusion-v2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o Inkpunk-Diffusion-v2.ckpt
elif Model=="Izumi v1":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/iz/resolve/main/izumi_01.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o izumi_01.ckpt
elif Model=="Knollingcase":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/knollingcase/resolve/main/knollingcase.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o knollingcase.ckpt
elif Model=="Midjourney v4":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/midjourney-v4-diffusion/resolve/main/mdjrny-v4.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o mdjrny-v4.ckpt
elif Model=="Mo Di Diffusion":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/mo-di-diffusion/resolve/main/moDi-v1-pruned.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o moDi-v1-pruned.ckpt
elif Model=="Monstermash Any 3":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/monstermash_anyv3/resolve/main/monstermash4%2Banyv3.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o monstermash4%2Banyv3.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/monstermash_anyv3/resolve/main/monstermash6%2Banyv3.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o monstermash6%2Banyv3.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/anything-v3.0/resolve/main/Anything-V3.0.vae.pt -d /content/stable-diffusion-webui/models/Stable-diffusion -o Anything-V3.0-pruned.vae.pt
elif Model=="Mousey Mix":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/closertodeath/mouseymix/resolve/main/mouseymix.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o mouseymix.safetensors
elif Model=="Neverending Dream":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/11925?type=Pruned%20Model&format=SafeTensor" -d /content/stable-diffusion-webui/models/Stable-diffusion -o neverending_dream.safetensors
elif Model=="Open Anime Journey":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/OpenAnimeJourney/resolve/main/OpenAnimeJourney.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o OpenAnimeJourney.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/anything-v3.0/resolve/main/Anything-V3.0.vae.pt -d /content/stable-diffusion-webui/models/Stable-diffusion -o Anything-V3.0.vae.pt
elif Model=="Open Journey v2":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/openjourney-v2/resolve/main/openjourney-v2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o openjourney-v2.ckpt
elif Model=="Papercut Diffusion":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/Stable_Diffusion_PaperCut_Model/resolve/main/PaperCut_v1.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o PaperCut_v1.ckpt
elif Model=="Pastel Mix Stylized Anime":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/andite/pastel-mix/resolve/main/pastelmix-fp16.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o pastelmix-fp16.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o pastelmix-fp16.vae.pt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/andite/pastel-mix/resolve/main/pastelmix-fp32.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o pastelmix-fp32.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o pastelmix-fp32.vae.pt
elif Model=="Photorealostic Fuen v1":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/photorealistic-fuen-v1/resolve/main/photorealistic-fuen-v1.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o photorealistic-fuen-v1.ckpt
elif Model=="Pix2Pix":
    !git clone https://github.com/Klace/stable-diffusion-webui-instruct-pix2pix /content/stable-diffusion-webui/extensions/stable-diffusion-webui-instruct-pix2pix
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/timbrooks/instruct-pix2pix/resolve/main/instruct-pix2pix-00-22000.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o instruct-pix2pix-00-22000.safetensors
elif Model=="Pony Diffusion 2":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/pony-diffusion-v2/resolve/main/pony-diffusion-v2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o pony-diffusion-v2.ckpt
elif Model=="Popup Book Diffusion":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/popupBook-diffusion/resolve/main/popupBook-diffusion-v1.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o popupBook-diffusion-v1.ckpt
elif Model=="Protogen v2.2":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/Protogen_V2.2/resolve/main/Protogen_V2.2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o Protogen_V2.2.ckpt
elif Model=="Protogen X 3.4":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/darkstorm2150/Protogen_x3.4_Official_Release/resolve/main/ProtoGen_X3.4.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o Protogen_X3.4.ckpt
elif Model=="Protogen Nova":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/Protogen_Nova_Official_Release/resolve/main/Protogen_Nova_Official_Release.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o Protogen_Nova_Official_Release.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/Protogen_Nova_Official_Release/raw/main/Protogen_Nova_Official_Release.yaml -d /content/stable-diffusion-webui/models/Stable-diffusion -o Protogen_Nova_Official_Release.yaml
elif Model=="Rando mix 3":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/RandoMix3/resolve/main/RandoMix3.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o RandoMix3.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/RandoMix3/resolve/main/RandoMix3.vae.pt -d /content/stable-diffusion-webui/models/Stable-diffusion -o RandoMix3.vae.pt
elif Model=="Riffusion v1":
    !git clone https://github.com/enlyth/sd-webui-riffusion /content/stable-diffusion-webui/extensions/sd-webui-riffusion
    !rm /content/stable-diffusion-webui/extensions/sd-webui-riffusion/install.py /content/stable-diffusion-webui/extensions/sd-webui-riffusion/requirements.txt
    !pip -q install pedalboard
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/riffusion-model-v1/resolve/main/riffusion-model-v1.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o riffusion-model-v1.ckpt
elif Model=="RPG v4":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Anashel/rpg/resolve/main/RPG-V4-Model-Download/RPG-v4.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o RPG-v4.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Anashel/rpg/raw/main/RPG-V4-Model-Download/RPG-v4.yaml -d /content/stable-diffusion-webui/models/Stable-diffusion -o RPG-v4.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Anashel/rpg/resolve/main/RPG-VAE-Current-Download/vae-ft-mse-840000-ema-pruned.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o RPG-v4.vae.pt
elif Model=="SamDoesArt Diffusion":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/SamDoesArt-V3/resolve/main/SamDoesArt-V3.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o SamDoesArt-V3.ckpt
elif Model=="SamDoesSexy Blend":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/samdoessexy-blend/resolve/main/samdoessexyBlend.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o samdoessexyBlend.safetensors
elif Model=="Seek Art Mega v1":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/seek.art_MEGA/resolve/main/seek_art_mega_v1.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o seek_art_mega_v1.ckpt
elif Model=="Simpmaker 3k":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/SimpMaker-3K1/resolve/main/SimpMaker%203k.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o SimpMaker%203k.ckpt
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/SimpMaker-3K1/resolve/main/SimpMaker%203k1.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o SimpMaker%203k1.ckpt
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/SimpMaker-3K1/resolve/main/SimpMaker%203k2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o SimpMaker%203k2.ckpt
    # !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/SimpMaker-3K1/resolve/main/SimpMaker%203k3.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o SimpMaker%203k3.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o SimpMaker%203k.vae.pt
elif Model=="Stable Diffusion 1.4":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd14/resolve/main/sd-v1-4.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o sd-v1-4.ckpt
elif Model=="Stable Diffusion 1.5":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o v1-5-pruned-emaonly.ckpt
elif Model=="Stable Diffusion 1.5 Inpainting":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/i15/resolve/main/sd-v1-5-inpainting.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o sd-v1-5-inpainting.ckpt
elif Model=="Stable Diffusion 2":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2/resolve/main/768-v-ema.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o 768-v-ema.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2/raw/main/v2-inference-v.yaml -d /content/stable-diffusion-webui/models/Stable-diffusion -o 768-v-ema.yaml
elif Model=="Stable Diffusion 2 Base":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2-base/resolve/main/512-base-ema.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o 512-base-ema.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2-base/raw/main/v2-inference.yaml -d /content/stable-diffusion-webui/models/Stable-diffusion -o 512-base-ema.yaml
elif Model=="Stable Diffusion 2 Depth":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2-depth/resolve/main/512-depth-ema.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o 512-depth-ema.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2-depth/raw/main/v2-midas-inference.yaml -d /content/stable-diffusion-webui/models/Stable-diffusion -o 512-depth-ema.yaml
    !sed -i -e '/emb = emb.unsqueeze(0)/a\                name = list(data.keys())[0]' /content/stable-diffusion-webui/modules/textual_inversion/textual_inversion.py
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/spaablauw/FloralMarble/resolve/main/FloralMarble-400.pt -d /content/stable-diffusion-webui/embeddings -o FloralMarble-400.pt
elif Model=="Stable Diffusion 2 Inpainting":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2-inpainting/resolve/main/512-inpainting-ema.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o 512-inpainting-ema.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2-inpainting/raw/main/v2-inpainting-inference.yaml -d /content/stable-diffusion-webui/models/Stable-diffusion -o 512-inpainting-ema.yaml
elif Model=="Stable Diffusion 2.1":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o v2-1_768-ema-pruned.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2-1/raw/main/v2-inference-v.yaml -d /content/stable-diffusion-webui/models/Stable-diffusion -o v2-1_768-ema-pruned.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/joachimsallstrom/Double-Exposure-Embedding/resolve/main/dblx.pt -d /content/stable-diffusion-webui/embeddings -o dblx.pt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Conflictx/AnimeScreencap/resolve/main/AnimeScreenCap.pt -d /content/stable-diffusion-webui/embeddings -o AnimeScreenCap.pt
elif Model=="Stable Diffusion 2.1 Base":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o v2-1_512-ema-pruned.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2-1-base/raw/main/v2-inference.yaml -d /content/stable-diffusion-webui/models/Stable-diffusion -o v2-1_512-ema-pruned.yaml
elif Model=="Synthwave Punk v2":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/SynthwavePunk-v2/resolve/main/SynthwavePunk-v2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o SynthwavePunk-v2.ckpt
elif Model=="Uber Realistic v1":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/urp/resolve/main/uberRealisticPornMer_urpMv1_ema.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o uberRealisticPornMer_urpMv1_ema.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o uberRealisticPornMer_urpMv1_ema.vae.pt
elif Model=="Van Gogh Diffusion":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/Van-Gogh-diffusion/resolve/main/Van-Gogh-Style-lvngvncnt-v2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o Van-Gogh-Style-lvngvncnt-v2.ckpt
elif Model=="Vinte Protogen Mix":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/6582" -d /content/stable-diffusion-webui/models/Stable-diffusion -o vinteprotogenmix_v10.safetensors
elif Model=="Vintedois Diffusion":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/vintedois-diffusion-v0-1/resolve/main/vintedois_0_1.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o vintedois_0_1.ckpt
elif Model=="Waifu Diffusion":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float32.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o wd-v1-3-float32.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/waifu-diffusion-v1-4/resolve/main/kl-f8-anime2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o wd-v1-3-float32.vae.pt
elif Model=="Waifu Diffusion 1.4":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/waifu-diffusion-v1-4/resolve/main/wd-1-4-anime_e1.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o wd-1-4-anime_e1.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/waifu-diffusion-v1-4/raw/main/wd-1-4-anime_e1.yaml -d /content/stable-diffusion-webui/models/Stable-diffusion -o wd-1-4-anime_e1.yaml
elif Model=="Waifu Diffusion 1.5":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/wd-1-5-beta/resolve/main/wd15-beta1-fp32.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o wd15-beta1-fp32.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/wd-1-5-beta/raw/main/wd15-beta1-fp32.yaml -d /content/stable-diffusion-webui/models/Stable-diffusion -o wd15-beta1-fp32.yaml
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/waifu-diffusion-v1-4/resolve/main/kl-f8-anime2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o wd15-beta1-fp32.vae.pt
elif Model=="Woolitize":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/woolitize/resolve/main/woolitize.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o woolitize.ckpt
elif Model=="Yiffy E18":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/yiffy-e18/resolve/main/yiffy-e18.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o yiffy-e18.ckpt
elif Model=="Yiffy Mix":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/yiffymix/resolve/main/yiffymix.safetensors -d /content/stable-diffusion-webui/models/Stable-diffusion -o yiffymix.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/waifu-diffusion-v1-4/resolve/main/kl-f8-anime2.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o yiffymix.vae.pt
elif Model=="Yohan Diffusion":
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/yohan-diffusion/resolve/main/yohan-diffusion.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o yohan-diffusion.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/anything-v3.0/resolve/main/Anything-V3.0.vae.pt -d /content/stable-diffusion-webui/models/Stable-diffusion -o yohan-diffusion.vae.pt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/yohan-diffusion/resolve/main/Cocoa.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o Cocoa.ckpt
else:          
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o v1-5-pruned-emaonly.ckpt

## Downloading requested embeddings

if bad_artist:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/nick-x-hacker/bad-artist/resolve/main/bad-artist.pt -d /content/stable-diffusion-webui/embeddings -o bad-artist.pt
if bad_artist_anime:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/nick-x-hacker/bad-artist/resolve/main/bad-artist-anime.pt -d /content/stable-diffusion-webui/embeddings -o bad-artist-anime.pt
if bad_hands_5:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yesyeahvh/bad-hands-5/resolve/main/bad-hands-5.pt -d /content/stable-diffusion-webui/embeddings -o bad-hands-5.pt
if bad_prompt:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/Nerfgun3/bad_prompt/resolve/main/bad_prompt.pt -d /content/stable-diffusion-webui/embeddings -o bad_prompt.pt
if bad_prompt_version2:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/Nerfgun3/bad_prompt/resolve/main/bad_prompt_version2.pt -d /content/stable-diffusion-webui/embeddings -o bad_prompt_version2.pt

## Downloading requested VAE

if Anything_V3_vae:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Maseshi/Anything-v3.0/resolve/main/Anything-V3.0.vae.pt -d /content/stable-diffusion-webui/models/VAE -o Anything-V3.0.vae.pt
if vae_ft_mse_840000_ema_pruned:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -d /content/stable-diffusion-webui/models/VAE -o vae-ft-mse-840000-ema-pruned.safetensors

## Fixing ControlNet/Deforum "folder not found"

!mkdir /content/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui/models

## Launching environment

!python launch.py --share --xformers --enable-insecure-extension-access --gradio-queue $host $dark

In [ ]:
#@title Reconnect
#@markdown In case the above cell stop running, if the webui loses connection to Colab, or if you unintentionally refresh this page, use this cell to re-enact your environment without reinstalling everything. You will find your work intact.

# Moving in correct folder

%cd /content/stable-diffusion-webui

## Needed here is Host is Google

if host = "--googleusercontent":
    os.environ['colab_url'] = eval_js("google.colab.kernel.proxyPort(7860), {'cache': false}")

## Launching environment

!python launch.py --share --xformers --enable-insecure-extension-access --gradio-queue $host $dark